In [22]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import json

# Load the pre-trained model
model = load_model('updated94.h5')

image_dir = 'Registered Criminals'

# Load the existing label mapping
with open('labelCriminals.json', 'r') as f:
    labelCriminals = json.load(f)
print(labelCriminals)

# Freeze the last layer
model.layers[-1].trainable = False

# Add a new output layer with a unique name
num_classes = len(os.listdir(image_dir))
output = Dense(num_classes, activation='softmax', name='new_output')(model.layers[-1].output)

# Create a new model with the added output layer
new_model = Model(inputs=model.input, outputs=output)

# Compile the model with optimizer and loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up the data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define the image directory and iterate over the subfolders
for folder_name in os.listdir(image_dir):
    folder_path = os.path.join(image_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue  # Skip if it's not a subfolder

    # Collect the images from the original subfolder
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image = cv2.resize(image, (224, 224))  # Resize if necessary
        images.append(image)
        labels.append(folder_name)

        # Perform data augmentation and save augmented images
        augmented_images = datagen.flow(np.expand_dims(image, axis=0), batch_size=1)
        count = 0  # Counter for augmented images per original image
        for augmented_image in augmented_images:
            augmented_image = augmented_image[0].astype(np.uint8)
            count += 1
            augmented_image_path = os.path.join(folder_path, f'{filename}_{count}.jpg')
            cv2.imwrite(augmented_image_path, augmented_image)
            if count >= 200:
                break  # Stop augmenting after reaching 200 images per original image

    # Convert the image and label lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    # One-hot encode the labels
    unique_labels = np.unique(labels)
    # Update the label_mapping dictionary with new labels
    for label in unique_labels:
        if label not in label_mapping:
            label_mapping[label] = len(label_mapping)

    labels_encoded = np.array([label_mapping[label] for label in labels])
    labels_onehot = to_categorical(labels_encoded, num_classes=model.output_shape[-1])

    # Perform additional preprocessing if necessary

    # Train the model using the augmented images and labels
    model.fit(images, labels_onehot, batch_size=32, epochs=10)

    # Delete the imported images
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        os.remove(file_path)
    os.rmdir(folder_path)

# Save the updated model
model.save('updated94.h5')

# Save the updated label mapping to a JSON file
with open('labelCriminals.json', 'w') as f:
    json.dump(label_mapping, f)
print(label_mapping)


{'Aaron_Tippin': 0, 'Abba_Eban': 1, 'Abbas_Kiarostami': 2, 'Abdel_Aziz_Al-Hakim': 3, 'Abdel_Madi_Shabneh': 4, 'Abdel_Nasser_Assidi': 5, 'Abdoulaye_Wade': 6, 'Abdulaziz_Kamilov': 7, 'Abdullah': 8, 'Abdullah_Ahmad_Badawi': 9, 'Abdullah_al-Attiyah': 10, 'Abdullah_Gul': 11, 'Abdullah_Nasseef': 12, 'Abdullatif_Sener': 13, 'Abdul_Majeed_Shobokshi': 14, 'Abdul_Rahman': 15, 'Abel_Aguilar': 16, 'Abel_Pacheco': 17, 'Abid_Hamid_Mahmud_Al-Tikriti': 18, 'Abner_Martinez': 19, 'Abraham_Foxman': 20, 'Aby_Har-Even': 21, 'Bill_Kong': 22, 'Bill_Lerach': 23, 'Bill_Maher': 24, 'Bill_Mauldin': 25, 'Bill_McBride': 26, 'Bill_Nelson': 27, 'Bill_OReilly': 28, 'Bill_Parcells': 29, 'Bill_Parsons': 30, 'Bill_Paxton': 31, 'Bill_Pryor': 32, 'Bill_Rainer': 33, 'Bill_Readdy': 34, 'Kweisi_Mfume': 35, 'Kwon_Yang-sook': 36, 'Kwon_Young-gil': 37, 'Kyle_McLaren': 38, 'Kyle_Shewfelt': 39, 'Kyoko_Nakayama': 40, 'Kyra_Sedgwick': 41, 'Lachlan_Murdoch': 42, 'Laila_Ali': 43, 'Lana_Clarkson': 44, 'Lance_Armstrong': 45, 'Lance_Bas

IndexError: index 162 is out of bounds for axis 1 with size 162